[🥭 Entropy Markets](https://entropy.markets/) support is available at: [Docs](https://docs.entropy.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Deposit and Withdraw

This notebook shows how to deposit a quantity of a specific token into a Entropy Account, and then withdraw it.

This involves using Entropy's root and node banks. Root banks keep track of tokens deposited across all Entropy accounts, and node banks hold the actual tokens themselves. Much of this is handled for you by the `Account` object but it's possible to perform all the steps manually if you need to alter parts of the process.

In [ ]:
import decimal
import entropy


# Use this function to display balances because it always fetches fresh data
def show_balances(context, wallet, group, sol_token):
    # Find the current SPL token balance
    token_account = entropy.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
    print("Wrapped SOL in wallet:", token_account.value)

    # Find the SOL-specific details in the account
    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
    for slot in accounts[0].slots:
        if slot.base_instrument == sol_token:
            print("Wrapped SOL in account", slot.net_value)
            print()
            return


# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Signers are effectively an empty CombinableInstruction that only carries the keys for
# signing transactions
signers: entropy.CombinableInstructions = entropy.CombinableInstructions.from_wallet(wallet)

# Create a 'mainnet' Context
with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    # Entropy accounts are per-Group, so we need to load the Group first.
    group = entropy.Group.load(context)

    # Find the wrapped SOL token
    sol_token = context.instrument_lookup.find_by_symbol_or_raise("SOL")

    print("Before:")
    show_balances(context, wallet, group, sol_token)

    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    print("Depositing 0.5 SOL:")

    deposit_value = entropy.InstrumentValue(sol_token, decimal.Decimal("0.5"))
    deposit_signatures = account.deposit(context, wallet, deposit_value)
    print("Deposit transaction signatures:", deposit_signatures)

    print("Waiting for deposit transaction to confirm...\n")
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, deposit_signatures, commitment="processed"
        )

    print("After deposit:")
    show_balances(context, wallet, group, sol_token)

    print("Withdrawing 0.5 SOL:")

    withdrawal_value = entropy.InstrumentValue(sol_token, decimal.Decimal("0.5"))
    withdrawal_signatures = account.withdraw(context, wallet, withdrawal_value, False)
    print("Withdrawal transaction signatures:", withdrawal_signatures)

    print("Waiting for withdraw transaction to confirm...\n")
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, withdrawal_signatures, commitment="processed"
        )

    print("After withdrawal:")
    show_balances(context, wallet, group, sol_token)

print("Example complete.")